In [1]:
import pandas as pd
import numpy as np
import os
import math

import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
root_dir = "/nfs/turbo/umms-indikar/shared/projects/live_cell_imaging/6A/subset_outputs/B5/"

MINUTES_PER_FRAME = 25


for _dir, condition in experiments.items():
    tracks_path = f"{root_dir}{_dir}/Track statistics.csv"
    tracks = pd.read_csv(tracks_path)
    
    spots_path = f"{root_dir}{_dir}/All Spots statistics.csv"
    spots = pd.read_csv(spots_path)
    
    
    print(condition, spots['TRACK_ID'].nunique())
    
    # convert types
    spots = spots[spots['TRACK_ID'] != "None"]
    spots = spots.astype({"TRACK_ID": int})
    
    tracks = tracks.astype({"TRACK_ID": int})
    
    # merge 
    tmp = pd.merge(spots, tracks, how='left', on=['TRACK_ID'])
    
    # sort by ID then FRAME
    tmp = tmp.sort_values(by=['TRACK_ID', 'FRAME'])

    # add a grouped step 
    tmp.loc[:, 'STEP'] =  tmp.groupby(['TRACK_ID']).cumcount()
    tmp.loc[:, 'NORMED_STEP'] = tmp['STEP'] / tmp['TRACK_DURATION']
    
    tmp['Scene'] = _dir
    tmp['Condition'] = condition
    tmp['Time'] = (tmp['FRAME'] * MINUTES_PER_FRAME) / 60
    
    tmp['TRACK_ID'] = tmp['TRACK_ID'].astype(str) + "_" + str(_dir)
    df_list.append(tmp)
    
    
df = pd.concat(df_list, ignore_index=True)

df.head()